In [1]:
!pip install -U autogluon > /dev/null
from autogluon.tabular import TabularDataset, TabularPredictor

import numpy as np
import pandas as pd
from pandas import DataFrame, Series
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from optuna import Trial
from optuna import create_study
import optuna
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.base import BaseEstimator, TransformerMixin

import xgboost as xgb
import lightgbm as lgb



ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
keras-cv 0.8.2 requires keras-core, which is not installed.
keras-nlp 0.8.1 requires keras-core, which is not installed.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
aiobotocore 2.11.2 requires botocore<1.34.35,>=1.33.2, but you have botocore 1.29.165 which is incompatible.
albumentations 1.4.0 requires scikit-image>=0.21.0, but you have scikit-image 0.20.0 which is incompatible.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 15.0.0 which is incompatible.
beatrix-jupyterlab 2023.128.151533 requires jupyterlab~=3.6.0, but you have jupyterlab 4.1.2 which is incompa

In [2]:
train_b=pd.read_csv('/kaggle/input/playground-series-s4e2/train.csv')
test=pd.read_csv('/kaggle/input/playground-series-s4e2/test.csv')
train_org=pd.read_csv('/kaggle/input/obesity-or-cvd-risk-classifyregressorcluster/ObesityDataSet.csv')

In [3]:
train = pd.concat([train_b, train_org], ignore_index=True)
train = train.drop_duplicates()
y_train = train['NObeyesdad']

In [4]:
X, X_val, y, y_val = train_test_split(train, y_train,test_size=0.2, random_state=42)
X['NObeyesdad']=y

In [5]:
train =  TabularDataset(X)
test = TabularDataset(test)

In [6]:
target_col = 'NObeyesdad'
# Define the models to use
hyperparameters = {
    'GBM': [{}],  # Use default hyperparameters for LightGBM
    'CAT': [{}],  # Use default hyperparameters for CatBoost
    'XGB': [{}],
    'RF': [{}],
    'XT': [{}]
}


In [7]:
automl = TabularPredictor(label=target_col, problem_type='multiclass', eval_metric='accuracy')
automl.fit(train, presets='best_quality', hyperparameters=hyperparameters)

No path specified. Models will be saved in: "AutogluonModels/ag-20240229_074352"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Dynamic stacking is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
Detecting stacked overfitting by sub-fitting AutoGluon on the input data. That is, copies of AutoGluon will be sub-fit on subset(s) of the data. Then, the holdout validation data is used to detect stacked overfitting.
Sub-fit(s) time limit is: 3600 seconds.
Starting holdout-based sub-fit for dynamic stacking. Context path is: AutogluonModels/ag-20240229_074352/ds_sub_fit/sub_fit_ho.
2024-02-29 07:43:53,450	INFO util.py:129 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
Beginning Au

In [8]:
def apply_thresholds(y_proba, thresholds):
    # Apply the specified thresholds to convert probabilities to predicted labels
    y_pred_labels = np.argmax(y_proba, axis=1)
    for i in range(y_proba.shape[1]):
        y_pred_labels[y_proba[:, i] > thresholds[f'threshold_{i}']] = i

    return y_pred_labels

def objective(trial : Trial, num_classes : int, pred_proba, y_val):
    # Define the thresholds for each class
    thresholds = {}
    for i in range(num_classes):
        thresholds[f'threshold_{i}'] = trial.suggest_float(f'threshold_{i}', 0.0, 1.0)

    # Apply the thresholds to convert probabilities to predictions
    y_pred = apply_thresholds(pred_proba, thresholds)

    # Calculate accuracy
    accuracy = accuracy_score(y_val, y_pred)
    return accuracy

In [9]:
class TargetEncoder(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.encoder = OrdinalEncoder(categories=[['Insufficient_Weight',
                                                   'Normal_Weight',
                                                   'Overweight_Level_I',
                                                   'Overweight_Level_II',
                                                   'Obesity_Type_I',
                                                   'Obesity_Type_II',
                                                   'Obesity_Type_III']])

    def fit(self, y):
        self.encoder.fit(y)
        return self

    def transform(self, y):
        return self.encoder.transform(y)

    def inverse_transform(self, y):
        return self.encoder.inverse_transform(y)
    

In [10]:
num_classes = 7
pred_proba = automl.predict_proba(X_val).to_numpy()  # Example: replace with actual y_pred_proba
encoder = TargetEncoder()

y_val = encoder.fit_transform(pd.DataFrame(y_val))  # Example: replace with actual y_val

In [ ]:
study2 = optuna.create_study(direction='maximize')
study2.optimize(lambda trial: objective(trial, num_classes, pred_proba, y_val), n_trials=500)

# # Get the best thresholds
best_thresholds = study2.best_params
print("Best Thresholds:", best_thresholds)

[I 2024-02-29 08:35:45,011] A new study created in memory with name: no-name-34d56839-47e2-48d6-bddf-1e4e89862419
[I 2024-02-29 08:35:45,021] Trial 0 finished with value: 0.2460056905230904 and parameters: {'threshold_0': 0.8807685537145873, 'threshold_1': 0.8740679052732361, 'threshold_2': 0.22286572708643804, 'threshold_3': 0.8568815794834631, 'threshold_4': 0.3215936627623165, 'threshold_5': 0.5606611134736597, 'threshold_6': 0.07706375409769384}. Best is trial 0 with value: 0.2460056905230904.
[I 2024-02-29 08:35:45,027] Trial 1 finished with value: 0.24972641715911578 and parameters: {'threshold_0': 0.9319857787433327, 'threshold_1': 0.5000189363056621, 'threshold_2': 0.61539896015671, 'threshold_3': 0.7131018723579464, 'threshold_4': 0.18657432885363578, 'threshold_5': 0.3820232332557354, 'threshold_6': 0.25299403463318504}. Best is trial 1 with value: 0.24972641715911578.
[I 2024-02-29 08:35:45,033] Trial 2 finished with value: 0.2506018822499453 and parameters: {'threshold_0': 

In [18]:
test_label = automl.predict_proba(test).to_numpy()

test_label = apply_thresholds(test_label, best_thresholds)

In [19]:
test_label = automl.predict(test)

In [22]:
submission = pd.read_csv('/kaggle/input/playground-series-s4e2/sample_submission.csv')
submission['NObeyesdad'] = test_label

In [23]:
submission.to_csv('submission.csv', index=False)